<a href="https://colab.research.google.com/github/feiyoung/ReadPapers/blob/master/GNN_learn_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Attributed Graph neural network

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class GraphConvolutionLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolutionLayer, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
        torch.nn.init.xavier_uniform_(self.weight)

    def forward(self, x, adj_matrix):
        support = torch.mm(x, self.weight)
        output = torch.mm(adj_matrix, support) + self.bias
        return output

class GraphNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GraphNeuralNetwork, self).__init__()
        self.gc1 = GraphConvolutionLayer(input_size, hidden_size)
        self.gc2 = GraphConvolutionLayer(hidden_size, output_size)

    def forward(self, x, adj_matrix):
        x = F.relu(self.gc1(x, adj_matrix))
        x = self.gc2(x, adj_matrix)
        return x

    def get_node_embeddings(self, x, adj_matrix):
        # This method returns the node embeddings without the final activation function
        x = F.relu(self.gc1(x, adj_matrix))
        x = self.gc2(x, adj_matrix)
        return x



In [7]:
# Example usage
# Assuming you have an adjacency matrix (adj_matrix) and node features (x)

# Hyperparameters
input_size = 3  # Number of features for each node
hidden_size = 64
output_size = 16  # Adjust based on your task

# Create an instance of the GraphNeuralNetwork
gnn = GraphNeuralNetwork(input_size, hidden_size, output_size)

# Example adjacency matrix (replace this with your actual adjacency matrix)
adj_matrix = torch.tensor([[0.0, 0.5, 0.2],
                          [0.5, 0.0, 0.8],
                          [0.2, 0.8, 0.0]])

# Example node features (replace this with your actual node features)
x = torch.tensor([[1.0, 2.0, 3.0],
                  [4.0, 5.0, 6.0],
                  [7.0, 8.0, 9.0]])

# Forward pass
output = gnn(x, adj_matrix)

# Get node embeddings
node_embeddings = gnn.get_node_embeddings(x, adj_matrix)



tensor([[ 9.7266e+01,  8.5493e+20, -1.6717e+02,  3.3610e+21,  1.2256e+02,
          7.9884e+20, -1.7684e+02, -1.0007e+02, -5.8829e+00, -2.5528e+02,
         -1.5681e+02,  4.5447e+30,  7.0062e+22,  4.7899e+01,  4.5447e+30,
          7.0062e+22],
        [ 1.7983e+02,  8.5493e+20, -3.0936e+02,  3.3610e+21,  2.2746e+02,
          7.9884e+20, -3.2813e+02, -1.8593e+02, -8.8979e+00, -4.7509e+02,
         -2.9133e+02,  4.5447e+30,  7.0062e+22,  8.9437e+01,  4.5447e+30,
          7.0062e+22],
        [ 1.3900e+02,  8.5493e+20, -2.3887e+02,  3.3610e+21,  1.7511e+02,
          7.9884e+20, -2.5262e+02, -1.4296e+02, -8.5334e+00, -3.6462e+02,
         -2.2401e+02,  4.5447e+30,  7.0062e+22,  6.8421e+01,  4.5447e+30,
          7.0062e+22]], grad_fn=<AddBackward0>)


In [ ]:
print(node_embeddings)

2. Use reconstruction error as the loss to train the GAT

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class GraphConvolutionLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolutionLayer, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
        torch.nn.init.xavier_uniform_(self.weight)

    def forward(self, x, adj_matrix):
        support = torch.mm(x, self.weight)
        output = torch.mm(adj_matrix, support) + self.bias
        return output

class GraphAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(GraphAutoencoder, self).__init__()
        self.gc_encoder = GraphConvolutionLayer(input_size, hidden_size)
        self.gc_decoder = GraphConvolutionLayer(hidden_size, input_size)

    def forward(self, x, adj_matrix):
        # Encoder
        encoded = F.relu(self.gc_encoder(x, adj_matrix))

        # Decoder
        decoded = self.gc_decoder(encoded, adj_matrix.t())  # Assuming undirected graph

        return encoded, decoded



In [11]:
# Example usage
# Assuming you have an adjacency matrix (adj_matrix) and node features (x)

# Hyperparameters
input_size = 3  # Number of features for each node
hidden_size = 2
learning_rate = 0.001
num_epochs = 100

# Create an instance of the GraphAutoencoder
autoencoder = GraphAutoencoder(input_size, hidden_size)
optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
# Define the reconstruction loss function
def reconstruction_loss(Z, A):
    return torch.sum((torch.mm(Z, Z.t()) - torch.mm(A, A.t()))**2)

# Example adjacency matrix (replace this with your actual adjacency matrix)
adj_matrix = torch.tensor([[0.0, 0.5, 0.2],
                          [0.5, 0.0, 0.8],
                          [0.2, 0.8, 0.0]])

# Example node features (replace this with your actual node features)
x = torch.tensor([[1.0, 2.0, 3.0],
                  [4.0, 5.0, 6.0],
                  [7.0, 8.0, 9.0]])

# Training the model
for epoch in range(num_epochs):
    optimizer.zero_grad()
    encoded, decoded = autoencoder(x, adj_matrix)

    # Reconstruction loss for adjacency matrix
    adj_reconstruction_loss = reconstruction_loss(decoded, adj_matrix)

    # Reconstruction loss for node features
    attribute_reconstruction_loss = reconstruction_loss(encoded, x)

    # Total loss
    total_loss = adj_reconstruction_loss + attribute_reconstruction_loss

    # Backward pass and optimization
    total_loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Total Loss: {total_loss.item():.4f}')

# Reconstruction after training
encoded, decoded = autoencoder(x, adj_matrix)
print("Original Adjacency Matrix:")
print(adj_matrix)
print("Reconstructed Adjacency Matrix:")
print(decoded.detach().numpy())
print("Original Node Features:")
print(x)
print("Reconstructed Node Features:")
print(encoded.detach().numpy())


Epoch [10/100], Total Loss: 29697666711552.0000
Epoch [20/100], Total Loss: 29430309191680.0000
Epoch [30/100], Total Loss: 29166275657728.0000
Epoch [40/100], Total Loss: 28905698230272.0000
Epoch [50/100], Total Loss: 28648583200768.0000
Epoch [60/100], Total Loss: 28394848780288.0000
Epoch [70/100], Total Loss: 28144400596992.0000
Epoch [80/100], Total Loss: 27897108627456.0000
Epoch [90/100], Total Loss: 27652847042560.0000
Epoch [100/100], Total Loss: 27411523567616.0000
Original Adjacency Matrix:
tensor([[0.0000, 0.5000, 0.2000],
        [0.5000, 0.0000, 0.8000],
        [0.2000, 0.8000, 0.0000]])
Reconstructed Adjacency Matrix:
[[1288.0089       2.543558   123.26287  ]
 [1328.6759       3.942283   232.53368  ]
 [1308.0443       3.7181284  175.90778  ]]
Original Node Features:
tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])
Reconstructed Node Features:
[[243.27205     5.7657104]
 [244.15155    10.330714 ]
 [243.36494     6.2428837]]
